In [44]:
# All imports, just in case
%load_ext autoreload
%autoreload 2

from sympy import *
from sympy.tensor import *
import sympy.printing as printing

from variables import *
from structure import *
from functions import *
from latex     import *
from utils import read_obj, write_obj, subs_for_ode, subs_init, print_all_variables
import utils 
from ode_utils import numerical_solver
from pyodesys.symbolic import SymbolicSys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
# psi[0], psi[1], psi[2] = symbols('psi0, psi1, psi2')
# theta[0], theta[1], theta[2] = symbols('theta0, theta1, theta2')

In [46]:
eq['diff(psi)'] = read_obj('diff(psi)(nu)')
eq['diff(theta)'] = read_obj('diff(theta)(nu)')


In [47]:
eq['dot(psi)'] = [eq['diff(psi)'](0), eq['diff(psi)'](1), eq['diff(psi)'](2)]
eq['dot(theta)'] = [eq['diff(theta)'](0), eq['diff(theta)'](1), eq['diff(theta)'](2)]

In [48]:
for i in range(3):
    eq['dot(psi)'][i] = eq['dot(psi)'][i].subs({beta[0]: 0, beta[1]: 2*pi/3, beta[2]: 4*pi/3})
    eq['dot(theta)'][i] = eq['dot(theta)'][i].subs({beta[0]: 0, beta[1]: 2*pi/3, beta[2]: 4*pi/3})    

In [49]:
eq['dot(theta)']

[(-d*Derivative(alpha(t), t) - nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - cos(theta0(t))*Derivative(alpha(t), t))/d,
 (-d*Derivative(alpha(t), t) - nu1(t)*sin(theta1(t)) + nu2(t)*cos(theta1(t)) + cos(theta1(t) + pi/3)*Derivative(alpha(t), t))/d,
 (-d*Derivative(alpha(t), t) - nu1(t)*sin(theta2(t)) + nu2(t)*cos(theta2(t)) + sin(theta2(t) + pi/6)*Derivative(alpha(t), t))/d]

In [50]:
# x,y,alpha = symbol('x, y, alpha')

x = Function('x')(t)
y = Function('y')(t)
alpha = Function('alpha')(t)

p = symbols('p')

# dalpha = diff(alpha)
eq['x'] = t
eq['y'] = 0.0
eq['alpha'] = 3.0
eq['dot(alpha)'] = diff(eq['alpha']).doit()

# alpha = symbols('alpha')

In [51]:
eq['nu1'] = cos(alpha)*diff(x) - sin(alpha)*diff(y)
eq['nu2'] = sin(alpha)*diff(x) + cos(alpha)*diff(y)

In [52]:
for i in range(1,3):
    eq['nu' + str(i)] = eq['nu' + str(i)].subs({x: eq['x'], y: eq['y'], Derivative(alpha, t): eq['dot(alpha)'], alpha: eq['alpha']})

In [53]:
for i in range(3):
    eq['dot(psi)'][i] = eq['dot(psi)'][i].\
        subs({nu[1]: eq['nu1'], nu[2]: eq['nu2'], Derivative(alpha, t): eq['dot(alpha)'], alpha: eq['alpha']}).\
        doit()
    eq['dot(theta)'][i] = eq['dot(theta)'][i].\
        subs({nu[1]: eq['nu1'], nu[2]: eq['nu2'], Derivative(alpha, t): eq['dot(alpha)'], alpha: eq['alpha']}).\
        doit()  

In [54]:
psi0, psi1, psi2, theta0, theta1, theta2, x, y, alpha = symbols('psi0, psi1, psi2, theta0, theta1, theta2, x, y, alpha')
for i in range(3):
    eq['dot(psi)'][i] = eq['dot(psi)'][i].subs({x: symbols('x'),
                         y: symbols('y'),
                         alpha: symbols('alpha'),
                         psi[0]: symbols('psi0'),
                         psi[1]: symbols('psi1'),
                         psi[2]: symbols('psi2'),
                         theta[0]: symbols('theta0'),
                         theta[1]: symbols('theta1'),
                         theta[2]: symbols('theta2')
                        })

    eq['dot(theta)'][i] = eq['dot(theta)'][i].subs({x: symbols('x'),
                         y: symbols('y'),
                         alpha: symbols('alpha'),
                         psi[0]: symbols('psi0'),
                         psi[1]: symbols('psi1'),
                         psi[2]: symbols('psi2'),
                         theta[0]: symbols('theta0'),
                         theta[1]: symbols('theta1'),
                         theta[2]: symbols('theta2')
                        })

In [55]:
# eq['dot(psi)']

In [56]:
# eq['dot(theta)']

In [57]:
# eq['dot(psi)']

In [58]:
# eq['dot(psi)']

In [59]:
# eq['dot(theta)']

In [60]:
left = [psi0, psi1, psi2, theta0, theta1, theta2]

In [61]:
right = [eq['dot(psi)'][0], eq['dot(psi)'][1], eq['dot(psi)'][2], eq['dot(theta)'][0], eq['dot(theta)'][1], eq['dot(theta)'][2]]

In [62]:
print(right)

[-(0.141120008059867*sin(theta0) - 0.989992496600445*cos(theta0))/r, -(0.141120008059867*sin(theta1) - 0.989992496600445*cos(theta1))/r, -(0.141120008059867*sin(theta2) - 0.989992496600445*cos(theta2))/r, (0.989992496600445*sin(theta0) + 0.141120008059867*cos(theta0))/d, (0.989992496600445*sin(theta1) + 0.141120008059867*cos(theta1))/d, (0.989992496600445*sin(theta2) + 0.141120008059867*cos(theta2))/d]


In [63]:
print('Количество совпадает:', len(left) == len(right))

Количество совпадает: True


In [64]:
list(zip(left, right))

[(psi0, -(0.141120008059867*sin(theta0) - 0.989992496600445*cos(theta0))/r),
 (psi1, -(0.141120008059867*sin(theta1) - 0.989992496600445*cos(theta1))/r),
 (psi2, -(0.141120008059867*sin(theta2) - 0.989992496600445*cos(theta2))/r),
 (theta0, (0.989992496600445*sin(theta0) + 0.141120008059867*cos(theta0))/d),
 (theta1, (0.989992496600445*sin(theta1) + 0.141120008059867*cos(theta1))/d),
 (theta2, (0.989992496600445*sin(theta2) + 0.141120008059867*cos(theta2))/d)]

In [65]:

# for i in range(len(right)):
#     right[i] = right[i].subs({alpha: eq['alpha']})

In [66]:
# right[0].subs()

In [67]:
# ['psi0', 'psi1', 'psi2', 'theta0', 'theta1', 'theta2']
# initial_conditions = '{}, {}, {}, {}, {}, {}'.format(0, 0, 0, 0, 0, 0)
initial_conditions = {psi0:0, psi1:0, psi2:0, theta0:pi/2, theta1:0, theta2:0}
params = {p: 3, r: 1, d: 0.2, h:2}
t_end = 2*pi.evalf()

In [68]:
odesys = SymbolicSys(list(zip(left, right)), t, [p, r, d])
''' Создаем точки по t'''
# tout = np.linspace(tstart, tend, nt)
xout, yout, info = odesys.integrate(t_end,
                                    initial_conditions,
                                    params,
                                    integrator='gsl',
                                    method='rk8pd',
                                    atol=1e-11,
                                    rtol=1e-12)


In [69]:
import numpy as np

def subs_symbols(obj, yout, params=None):
    obj = obj.\
        subs(Function('alpha')(t), eq['alpha']).\
        subs(theta[0], symbols('theta0')).\
        subs(theta[1], symbols('theta1')).\
        subs(theta[2], symbols('theta2')).\
        subs(psi[0], symbols('psi0')).\
        subs(psi[1], symbols('psi1')).\
        subs(psi[2], symbols('psi2')).\
        subs({beta[0]: 0, beta[1]: 2*pi/3, beta[2]: 4*pi/3}).\
        subs({symbols('theta0'): yout[3]}).\
        subs({symbols('theta1'): yout[4]}).\
        subs({symbols('theta2'): yout[5]})
    if params:
        obj = obj.subs(params)
    return obj

def evaluate_two_vec(vec1, vec2, xout, yout, params):
    temp = [[], [], []]
    for i in range(3):
        for t_step, res_step in zip(xout, yout):
            temp[i].append(subs_symbols(vec1(i) + vec2(i), res_step, params).\
                           subs({t: t_step}).\
                           tolist())
    return temp

def evaluate_xy(eq_x, eq_y, xout):
    temp = {'x': [], 'y': []}
    for t_step in xout:
        if not type(eq_x) == float:
            temp['x'].append(eq_x.subs({t: t_step}))
        if not type(eq_y) == float:
            temp['y'].append(eq_y.subs({t: t_step}))
    return temp

def sum_two_list(a, b):
    return np.add(a, b)

def point_of_contact(x, y, params, ):
    pass

def np_concatenate(a):
    return list(np.concatenate(a))

def to_np_float(obj):
    obj = np.array(obj).astype(np.float64)
    return obj

In [70]:
# SP_coord = evaluate_vec(SP)
# PC_coord = evaluate_vec(PC)

In [71]:
# SC_coord = sum_two_list(SP_coord, PC_coord)
# SC_coord.shape

In [72]:
SC_coord = np.array(evaluate_two_vec(SP, PC, xout, yout, params))

In [73]:
# subs_symbols(SP(1), yout)[0]

In [74]:
# subs_symbols(e[1][0], yout)

In [75]:
# sum_two_list(SP(0), PC(0))

In [76]:
# evaluate_two_vec(SP, PC, xout)

In [77]:
# e.shape

In [78]:
# from copy import copy
# # print(SP_coord[0][0] + PC_coord[0][0])
# tmp1 = copy(SP_coord)
# tmp2 = copy(PC_coord)
# for x,y  in zip(tmp1, tmp2):
#     for 
# # print(len(SP_coord[0]))

In [79]:
import pandas as pd

df = pd.DataFrame(yout, index=xout, columns=['psi0', 'psi1', 'psi2', 'theta0', 'theta1', 'theta2'])



In [80]:
# SP(0) + PC(0)

In [81]:
df['contact0x'] = pd.Series(to_np_float(np_concatenate(SC_coord[0][:,0])), index=df.index)
df['contact0y'] = pd.Series(to_np_float(np_concatenate(SC_coord[0][:,1])), index=df.index)
df['contact1x'] = pd.Series(to_np_float(np_concatenate(SC_coord[1][:,0])), index=df.index)
df['contact1y'] = pd.Series(to_np_float(np_concatenate(SC_coord[1][:,1])), index=df.index)
df['contact2x'] = pd.Series(to_np_float(np_concatenate(SC_coord[2][:,0])), index=df.index)
df['contact2y'] = pd.Series(to_np_float(np_concatenate(SC_coord[2][:,1])), index=df.index)



In [82]:
df


,psi0,psi1,psi2,theta0,theta1,theta2,contact0x,contact0y,contact1x,contact1y,contact2x,contact2y
0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.570796,0.000000e+00,0.000000e+00,-1.018216,-0.056878,0.174784,-0.899695,0.419211,0.815023
2.220446e-14,-3.133494e-15,2.198225e-14,2.198225e-14,1.570796,1.566747e-14,1.566747e-14,-1.018216,-0.056878,0.174784,-0.899695,0.419211,0.815023
1.332268e-13,-1.880096e-14,1.318935e-13,1.318935e-13,1.570796,9.400481e-14,9.400481e-14,-1.018216,-0.056878,0.174784,-0.899695,0.419211,0.815023
6.883383e-13,-9.713830e-14,6.814497e-13,6.814497e-13,1.570796,4.856915e-13,4.856915e-13,-1.018216,-0.056878,0.174784,-0.899695,0.419211,0.815023
3.463896e-12,-4.888250e-13,3.429231e-12,3.429231e-12,1.570796,2.444125e-12,2.444125e-12,-1.018216,-0.056878,0.174784,-0.899695,0.419211,0.815023
1.734168e-11,-2.447259e-12,1.716814e-11,1.716814e-11,1.570796,1.223629e-11,1.223629e-11,-1.018216,-0.056878,0.174784,-0.899695,0.419211,0.815023
8.673062e-11,-1.223943e-11,8.586267e-11,8.586267e-11,1.570796,6.119713e-11,6.119713e-11,-1.018216,-0.056878,0.174784,-0.899695,0.419211,0.815023
4.336753e-10,-6.120026e-11,4.293353e-10,4.293353e-10,1.570796,3.060013e-10,3.060013e-10,-1.018216,-0.056878,0.174784,-0.899695,0.419211,0.815023
2.168399e-09,-3.060045e-10,2.146699e-09,2.146699e-09,1.570796,1.530022e-09,1.530022e-09,-1.018216,-0.056878,0.174784,-0.899695,0.419211,0.815023
1.084202e-08,-1.530026e-09,1.073351e-08,1.073351e-08,1.570796,7.650127e-09,7.650127e-09,-1.018216,-0.056878,0.174784,-0.899695,0.419211,0.815023


In [1]:
eq['nu']

NameError: name 'eq' is not defined

In [84]:
xy = evaluate_xy(eq['x'], eq['y'], xout)

In [85]:
df['x'] = to_np_float(xy['x'])
df['y'] = to_np_float(xy['y'])

ValueError: Length of values does not match length of index

In [ ]:
# to_np_float(xy['x'])

In [ ]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py

def data_for_plotly(*args):
#     plotly.tools.set_credentials_file(username='plotly-alice', api_key='vmspm9koyTQszVPthltY')
    data = []
    for arg in args:
        data.append(go.Scatter(
            x = arg['x'],
            y = arg['y'],
            mode = 'lines',
            name = arg['name']))

    return data

In [ ]:
#     {
#         'x': df.x + df.contact0x,
#         'y': df.y + df.contact0y,
#         'name': 'wheel 0'
#     },
#     {
#         'x': df.x + df.contact1x,
#         'y': df.y + df.contact1y,
#         'name': 'wheel 1'
#     },
#     {
#         'x': df.x + df.contact2x,
#         'y': df.y + df.contact2y,
#         'name': 'wheel 2'
#     }

In [ ]:
data = data_for_plotly(
    {
        'x': df.x.values,
        'y': df.y.values,
        'name': 'x y'
    },
    {
        'x': df.contact0x,
        'y': df.contact0y,
        'name': 'wheel 0'
    },
    {
        'x': df.contact1x,
        'y': df.contact1y,
        'name': 'wheel 1'
    },
)

df.x
layout = {
    'shapes': get_shapes(df, 50)
}


fig = {
    'data': data,
    'layout': layout,
}

py.iplot(fig, filename='line-mode')

In [ ]:
df.contact0x.iloc[50]

In [122]:
psi_eq = read_obj('diff(psi)_nu')(0)
theta_eq = read_obj('diff(theta)_nu')(0)

In [ ]:
all_eq = read_obj('all_eq')


all_eq['x'] = scalar(read_obj('point_S_velocity')(0), e['x']).subs(Derivative(psi[0],t), psi_eq).subs(Derivative(theta[0],t), theta_eq)
all_eq['y'] = scalar(read_obj('point_S_velocity')(0), e['y']).subs(Derivative(psi[0],t), psi_eq).subs(Derivative(theta[0],t), theta_eq)    

# Подставляем значения и избавляемся от массивов
for key, value in all_eq.items():
    all_eq[key] = subs_for_ode(subs_init(value))


In [130]:
nu1, nu2, alpha2, alpha1, psi0, theta0, psi1, theta1, psi2, theta2, x, y = symbols('nu1, nu2, alpha2, alpha1, psi0, theta0, psi1, theta1, psi2, theta2, x, y')

In [131]:
left = [nu1, nu2, alpha2, alpha1, psi0, theta0, psi1, theta1, psi2, theta2, x, y]

In [132]:
print('Переменные\n', all_eq.keys())
right = list(all_eq.values())

Переменные
 dict_keys(['nu1', 'nu2', 'alpha2', 'alpha1', 'psi0', 'theta0', 'psi1', 'theta1', 'psi2', 'theta2', 'x', 'y'])


In [133]:
print('Количество совпадает:', len(left) == len(right))

Количество совпадает: True


In [134]:
print('Переменные зависят от:\n')
for i in range(len(all_eq)):
    print(str(left[i]) + ': f', end='')
    print_all_variables(right[i])

Переменные зависят от:

nu1: f{theta2, nu2, theta1, theta0, alpha1, alpha2, nu1}
nu2: f{theta2, nu2, theta1, theta0, alpha1, alpha2, nu1}
alpha2: f{theta2, nu2, theta1, theta0, alpha2, nu1}
alpha1: f{alpha2}
psi0: f{theta0, nu2, alpha2, nu1}
theta0: f{theta0, nu2, alpha2, nu1}
psi1: f{nu1, nu2, alpha2, theta1}
theta1: f{nu1, nu2, alpha2, theta1}
psi2: f{theta2, nu2, alpha2, nu1}
theta2: f{theta2, nu2, alpha2, nu1}
x: f{nu2, theta0, alpha1, alpha2, nu1}
y: f{nu2, theta0, alpha1, alpha2, nu1}


In [135]:
pi_2 = float(pi/2)
mu = 1

In [136]:
p1, p2 = params = symbols('p1, p2', real=True)
params_value = [1., 10.]

# ['nu1', 'nu2', 'alpha2', 'alpha1', 'psi0', 'theta0', 'psi1', 'theta1', 'psi2', 'theta2', 'x', 'y']
initial_conditions = '1, 0, 0, 0, 0, {}, 0, {}, 0, {}, 0, 0'.format(pi_2*mu, pi_2*mu, pi_2*mu)

In [137]:
xout, yout, info = numerical_solver(left=left,
                 right=right,
                 y0=initial_conditions,
                 params=[], params_value=[],
                 savetxt='result.txt',
                 plot=True)

# print(info)

In [138]:
import pandas as pd

In [140]:
df = pd.DataFrame(data=yout, index=xout, columns=list(all_eq.keys()))
df

,nu1,nu2,alpha2,alpha1,psi0,theta0,psi1,theta1,psi2,theta2,x,y
0.000000,1.0,0.0,0.0,0.0,0.000000,1.570796,0.000000,1.570796,0.000000,1.570796,0.000000,0.0
0.101010,1.0,0.0,0.0,0.0,0.025464,1.469958,0.025464,1.469958,0.025464,1.469958,0.101010,0.0
0.202020,1.0,0.0,0.0,0.0,0.101344,1.370136,0.101344,1.370136,0.101344,1.370136,0.202020,0.0
0.303030,1.0,0.0,0.0,0.0,0.226139,1.272300,0.226139,1.272300,0.226139,1.272300,0.303030,0.0
0.404040,1.0,0.0,0.0,0.0,0.397478,1.177321,0.397478,1.177321,0.397478,1.177321,0.404040,0.0
0.505051,1.0,0.0,0.0,0.0,0.612292,1.085942,0.612292,1.085942,0.612292,1.085942,0.505051,0.0
0.606061,1.0,0.0,0.0,0.0,0.867016,0.998757,0.867016,0.998757,0.867016,0.998757,0.606061,0.0
0.707071,1.0,0.0,0.0,0.0,1.157797,0.916203,1.157797,0.916203,1.157797,0.916203,0.707071,0.0
0.808081,1.0,0.0,0.0,0.0,1.480688,0.838566,1.480688,0.838566,1.480688,0.838566,0.808081,0.0
0.909091,1.0,0.0,0.0,0.0,1.831811,0.765991,1.831811,0.765991,1.831811,0.765991,0.909091,0.0


In [93]:
import plotly.graph_objs as go
import plotly
import plotly.plotly as py

def data_for_plotly(*args):
    plotly.tools.set_credentials_file(username='plotly-alice', api_key='vmspm9koyTQszVPthltY')
    data = []
    for arg in args:
        data.append(go.Scatter(
            x = arg['x'],
            y = arg['y'],
            mode = 'lines',
            name = arg['name']))
    return data

In [141]:
data = data_for_plotly(
    {
        'x': df.index,
        'y': df.x,
        'name': 'x'
    },
    {
        'x': df.index,
        'y': df.nu1,
        'name': 'nu1(t)'
    },
    {
        'x': df.index,
        'y': df.nu2,
        'name': 'nu2(t)'
    },
    {
        'x': df.index,
        'y': df.theta1,
        'name': 'theta1(t)'
    },
        {
        'x': df.index,
        'y': df.alpha1,
        'name': 'alpha1(t)'
    }
)
py.iplot(data, filename='line-mode')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~plotly-alice/0 or inside your plot.ly account where it is named 'line-mode'


In [95]:
all_eq['nu1']

-(12*(-12*(-2*sin(alpha1) + 2*sin(alpha1 + pi/3) - 2*sin(alpha1 + theta0) - 2*sin(alpha1 + theta1) - 2*sin(alpha1 + theta2) - 2*cos(alpha1 + pi/6))*sin(alpha1) + 12*(-2*sin(alpha1 + pi/6) + 2*cos(alpha1) - 2*cos(alpha1 + pi/3) + 2*cos(alpha1 + theta0) + 2*cos(alpha1 + theta1) + 2*cos(alpha1 + theta2))*cos(alpha1))*sin(alpha1) + (144*sin(alpha1)**2 + 144*cos(alpha1)**2)*(-2*sin(alpha1) + 2*sin(alpha1 + pi/3) - 2*sin(alpha1 + theta0) - 2*sin(alpha1 + theta1) - 2*sin(alpha1 + theta2) - 2*cos(alpha1 + pi/6)))*(2*alpha2**2*sin(theta0)**3 - 4*alpha2**2*sin(theta0)*cos(theta0) - 8*alpha2**2*sin(theta0) - 2*alpha2**2*sin(theta1)**2*sin(theta1 + pi/3) - 2*sqrt(3)*alpha2**2*sin(theta1)*sin(theta1 + pi/3)*cos(theta1 + pi/3) + 4*sqrt(3)*alpha2**2*sin(theta1)*sin(theta1 + pi/3) - 4*alpha2**2*sin(theta1)*cos(theta1) + 2*alpha2**2*sin(theta2)**2*cos(theta2 + pi/6) - 2*sqrt(3)*alpha2**2*sin(theta2)*sin(theta2 + pi/6)*cos(theta2 + pi/6) - 4*alpha2**2*sin(theta2)*cos(theta2) + 4*sqrt(3)*alpha2**2*sin(th